# Backtest Strategy Comparison

This notebook demonstrates the backtesting engine by comparing all 10 implemented strategies across multiple assets.

## Overview

The backtesting engine uses Backtrader to simulate trading strategies with:
- Realistic commission schemes
- Position sizing
- Performance metrics (Sharpe, Sortino, Calmar, Max Drawdown, Kelly Criterion)

## Strategies Compared

1. **Rebalance**: Periodic portfolio rebalancing to target ratios
2. **NoRebalance**: Buy and hold (no rebalancing)
3. **SMACrossover**: Single SMA crossover timing
4. **SMACrossoverDouble**: Two SMA crossovers
5. **SMACrossoverTriple**: Three SMA crossovers
6. **MACDSingle**: MACD-based entry/exit
7. **MACDDual**: Dual MACD indicators
8. **DipBuySMA**: Buy dips relative to SMA
9. **DipBuyStdev**: Buy dips using standard deviation
10. **SMARebalMix**: Mixed SMA timing + rebalancing

In [ ]:
# Setup
import warnings

warnings.filterwarnings("ignore")

# Set environment
import os

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

os.environ["DYNACONF_ENV"] = "development"

from config import logger

logger.info("Notebook initialized")

## 1. Load Historical Data

Load price data for backtesting.

In [ ]:
from finbot.utils.data_collection_utils.yfinance.get_history import get_history

# Load test assets
test_assets = ["SPY", "TLT", "QQQ"]
asset_data = {}

for asset in test_assets:
    try:
        asset_data[asset] = get_history(asset, adjust_price=True)
        print(f"✓ Loaded {asset}: {len(asset_data[asset])} days")
    except Exception as e:
        print(f"✗ Error loading {asset}: {e}")

print(f"\nSuccessfully loaded {len(asset_data)} assets")

## 2. Import Strategies

Load all available strategies from the backtesting module.

In [ ]:
from finbot.services.backtesting.strategies.dip_buy_sma import DipBuySMA
from finbot.services.backtesting.strategies.dip_buy_stdev import DipBuyStdev
from finbot.services.backtesting.strategies.macd_dual import MACDDual
from finbot.services.backtesting.strategies.macd_single import MACDSingle
from finbot.services.backtesting.strategies.no_rebalance import NoRebalance
from finbot.services.backtesting.strategies.rebalance import Rebalance
from finbot.services.backtesting.strategies.sma_crossover import SMACrossover
from finbot.services.backtesting.strategies.sma_crossover_double import SMACrossoverDouble
from finbot.services.backtesting.strategies.sma_crossover_triple import SMACrossoverTriple
from finbot.services.backtesting.strategies.sma_rebal_mix import SMARebalMix

strategies = [
    ("Rebalance", Rebalance),
    ("NoRebalance", NoRebalance),
    ("SMACrossover", SMACrossover),
    ("SMACrossoverDouble", SMACrossoverDouble),
    ("SMACrossoverTriple", SMACrossoverTriple),
    ("MACDSingle", MACDSingle),
    ("MACDDual", MACDDual),
    ("DipBuySMA", DipBuySMA),
    ("DipBuyStdev", DipBuyStdev),
    ("SMARebalMix", SMARebalMix),
]

print(f"Loaded {len(strategies)} strategies for comparison")

## 3. Run Backtests

Execute backtests for each strategy on SPY (S&P 500).

In [ ]:
from finbot.services.backtesting.backtest_runner import BacktestRunner
from finbot.services.backtesting.compute_stats import compute_stats

backtest_results = []

for strategy_name, strategy_class in strategies:
    try:
        # Initialize backtest
        runner = BacktestRunner(
            strategy=strategy_class,
            data=asset_data["SPY"],
            cash=100000,
            commission=0.001,  # 0.1% commission
        )

        # Run backtest
        results = runner.run()

        # Compute statistics
        stats = compute_stats(portfolio_values=results["portfolio_value"], returns=results["returns"])

        backtest_results.append(
            {
                "Strategy": strategy_name,
                "Final Value": results["final_value"],
                "Total Return": results["total_return"],
                "CAGR": stats.get("cagr", np.nan),
                "Sharpe": stats.get("sharpe", np.nan),
                "Sortino": stats.get("sortino", np.nan),
                "Calmar": stats.get("calmar", np.nan),
                "Max Drawdown": stats.get("max_drawdown", np.nan),
                "Win Rate": stats.get("win_rate", np.nan),
            }
        )

        print(f"✓ Completed backtest for {strategy_name}")

    except Exception as e:
        print(f"✗ Error backtesting {strategy_name}: {e}")
        backtest_results.append(
            {
                "Strategy": strategy_name,
                "Final Value": np.nan,
                "Total Return": np.nan,
                "CAGR": np.nan,
                "Sharpe": np.nan,
                "Sortino": np.nan,
                "Calmar": np.nan,
                "Max Drawdown": np.nan,
                "Win Rate": np.nan,
            }
        )

results_df = pd.DataFrame(backtest_results)
print("\nBacktest Results Summary (SPY):")
print("=" * 120)
print(results_df.round(4))

## 4. Visualize Performance Comparison

Create comparative visualizations of key performance metrics.

In [ ]:
# Create bar charts for key metrics
fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=["Total Return", "Sharpe Ratio", "Max Drawdown", "Win Rate"],
    vertical_spacing=0.15,
    horizontal_spacing=0.12,
)

# Total Return
fig.add_trace(
    go.Bar(x=results_df["Strategy"], y=results_df["Total Return"], name="Total Return", marker_color="lightblue"),
    row=1,
    col=1,
)

# Sharpe Ratio
fig.add_trace(
    go.Bar(x=results_df["Strategy"], y=results_df["Sharpe"], name="Sharpe", marker_color="lightgreen"), row=1, col=2
)

# Max Drawdown (absolute value)
fig.add_trace(
    go.Bar(x=results_df["Strategy"], y=results_df["Max Drawdown"].abs(), name="Max DD", marker_color="salmon"),
    row=2,
    col=1,
)

# Win Rate
fig.add_trace(
    go.Bar(x=results_df["Strategy"], y=results_df["Win Rate"], name="Win Rate", marker_color="gold"), row=2, col=2
)

fig.update_layout(height=800, title_text="Strategy Performance Comparison (SPY Backtest)", showlegend=False)

fig.update_xaxes(tickangle=45)
fig.update_yaxes(title_text="Return (%)", row=1, col=1)
fig.update_yaxes(title_text="Sharpe Ratio", row=1, col=2)
fig.update_yaxes(title_text="Max DD (%)", row=2, col=1)
fig.update_yaxes(title_text="Win Rate (%)", row=2, col=2)

fig.show()

## 5. Rank Strategies by Different Metrics

Identify top performers across different performance dimensions.

In [ ]:
# Rank by different metrics
print("Top 3 Strategies by Total Return:")
print(results_df.nlargest(3, "Total Return")[["Strategy", "Total Return"]])
print()

print("Top 3 Strategies by Sharpe Ratio:")
print(results_df.nlargest(3, "Sharpe")[["Strategy", "Sharpe"]])
print()

print("Top 3 Strategies by Calmar Ratio:")
print(results_df.nlargest(3, "Calmar")[["Strategy", "Calmar"]])
print()

print("Top 3 Strategies by Win Rate:")
print(results_df.nlargest(3, "Win Rate")[["Strategy", "Win Rate"]])

## 6. Risk-Return Scatter Plot

Visualize the risk-return tradeoff across all strategies.

In [ ]:
# Create scatter plot of returns vs max drawdown
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=results_df["Max Drawdown"].abs(),
        y=results_df["Total Return"],
        mode="markers+text",
        text=results_df["Strategy"],
        textposition="top center",
        marker=dict(
            size=results_df["Sharpe"] * 10,  # Size by Sharpe ratio
            color=results_df["Sharpe"],
            colorscale="Viridis",
            showscale=True,
            colorbar=dict(title="Sharpe Ratio"),
        ),
        name="Strategies",
    )
)

fig.update_layout(
    title="Risk-Return Profile (Bubble Size = Sharpe Ratio)",
    xaxis_title="Max Drawdown (%)",
    yaxis_title="Total Return (%)",
    height=600,
    hovermode="closest",
)

fig.show()

## Key Findings

From comparing all 10 strategies on SPY:

1. **Buy and Hold Performance**: NoRebalance (buy and hold) often serves as a strong baseline, difficult to beat consistently

2. **Rebalancing Benefits**: Periodic rebalancing can improve risk-adjusted returns (Sharpe ratio) even if absolute returns are similar

3. **Technical Indicators**: SMA and MACD strategies can reduce drawdowns during market downturns but may underperform in strong bull markets

4. **Dip Buying**: Strategies that buy dips can outperform during volatile markets but require careful parameter tuning

5. **Risk-Adjusted Returns**: Highest absolute returns don't always mean best risk-adjusted performance (Sharpe/Sortino/Calmar ratios)

6. **Win Rate vs Returns**: High win rates don't guarantee high returns - a few large wins can outweigh many small losses

7. **Strategy Complexity**: More complex strategies (triple SMA, dual MACD) don't necessarily outperform simpler approaches

## Next Steps

- Test strategies on different assets (QQQ, bonds, commodities)
- Optimize strategy parameters using grid search
- Run walk-forward analysis to test robustness
- Compare performance in bull vs bear markets
- Add transaction costs and slippage modeling
- Test portfolio strategies combining multiple approaches